In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from urllib.parse import quote_plus
import requests
import os
import time
import re  # Import for sanitizing file names

# Path to your Excel file containing product names
input_excel_path = r"Scotland_MA.xlsx"  # Update with your actual file path

# Directory to save downloaded PDFs
output_folder = r"Output"  # Update with your desired folder path
os.makedirs(output_folder, exist_ok=True)

# Read the product names from the Excel file
df = pd.read_excel(input_excel_path)



# Function to sanitize file names by replacing invalid characters
def sanitize_filename(filename):
    return re.sub(r'[<>:"/\\|?*\n]', '_', filename).strip()

# Initialize Selenium WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

for index, row in df.iterrows():
    product = row['Product Name']  # Adjust 'Product Name' to your actual column name in the Excel file
    try:
        # URL-encode the product name
        encoded_product = quote_plus(product)
        search_url = f"https://products.mhra.gov.uk/search/?search={encoded_product}&page=1"

        # Open the search page with the encoded product name
        driver.get(search_url)

        print(f"Processing product: {product}")

        # Wait for the checkbox to appear and be clickable
        WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.ID, "agree-checkbox"))
        ).click()
        print("Checkbox clicked")

        # Wait for the "Agree" button to become enabled and click it
        WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Agree')]"))
        ).click()
        print("Agree button clicked")

        # Wait for the first result within the search results
        search_result = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div.search-result"))
        )

        # Find the first PDF link in the search results
        pdf_link = search_result.find_element(By.CSS_SELECTOR, "dd.right a").get_attribute("href")
        print(f"PDF Link: {pdf_link}")
    

        # Download the PDF and save it locally
        response = requests.get(pdf_link, stream=True)
        if response.status_code == 200:
            sanitized_name = sanitize_filename(product)
            pdf_filename = os.path.join(output_folder, f"{sanitized_name}.pdf")
            with open(pdf_filename, 'wb') as pdf_file:
                pdf_file.write(response.content)
            df.at[index, 'Download Status'] = 'Downloaded'
            print(f"PDF downloaded for product: {product}")
        else:
            df.at[index, 'Download Status'] = f"Failed: {response.status_code}"
            print(f"Failed to download PDF for product: {product}")

        # Add a delay to avoid getting blocked
        time.sleep(2)

    except Exception as e:
        df.at[index, 'PDF Link'] = f"Error: {e}"
        df.at[index, 'Download Status'] = 'Error'
        print(f"Error processing product {product}: {e}")

# Close the driver
driver.quit()


print("Finished processing all products.")


Processing product: MAGNEGITA 500 MICROMOL/ML SOLUTION FOR
INJECTION
Error processing product MAGNEGITA 500 MICROMOL/ML SOLUTION FOR
INJECTION: Message: 
Stacktrace:
	GetHandleVerifier [0x009533E3+25059]
	(No symbol) [0x008DCDE4]
	(No symbol) [0x007BBEC3]
	(No symbol) [0x007FFD86]
	(No symbol) [0x007FFFCB]
	(No symbol) [0x0083D952]
	(No symbol) [0x00821F44]
	(No symbol) [0x0083B51E]
	(No symbol) [0x00821C96]
	(No symbol) [0x007F3FAC]
	(No symbol) [0x007F4F3D]
	GetHandleVerifier [0x00C45543+3113795]
	GetHandleVerifier [0x00C5A20A+3198986]
	GetHandleVerifier [0x00C529E2+3168226]
	GetHandleVerifier [0x009F3250+680016]
	(No symbol) [0x008E572D]
	(No symbol) [0x008E29D8]
	(No symbol) [0x008E2B75]
	(No symbol) [0x008D57D0]
	BaseThreadInitThunk [0x76EA7BA9+25]
	RtlInitializeExceptionChain [0x77D9C0CB+107]
	RtlClearBits [0x77D9C04F+191]

Processing product: GADOPENTETATE CURAGITA 500
MICROMOL/ML SOLUTION FOR INJECTION
Error processing product GADOPENTETATE CURAGITA 500
MICROMOL/ML SOLUTION FOR